In [109]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error

In [110]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
holidays_events = pd.read_csv('holidays_events.csv')
oil = pd.read_csv('oil.csv')
stores = pd.read_csv('stores.csv')
transactions = pd.read_csv('transactions.csv')


In [111]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [112]:
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [113]:
holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [114]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [115]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [116]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [117]:
holidays_events.locale.unique()

array(['Local', 'Regional', 'National'], dtype=object)

In [118]:
# combine all datasets into a single datase

# oil -> train, test
oil.rename(columns={'dcoilwtico':'oil_price'}, inplace=True)
train = pd.merge(train, oil, on='date', how='left')
test = pd.merge(test, oil, on='date', how='left')

# stores -> train, test
train = pd.merge(train, stores, on='store_nbr', how='left')
test = pd.merge(test, stores, on='store_nbr', how='left')

# split holiday events into -> Local, Regional, National
condition = (holidays_events.transferred == True) | (holidays_events.type == 'Work Day')
special_days = holidays_events.drop(holidays_events[condition].index)
special_days = special_days.drop(['type', 'description', 'transferred'], axis=1)
special_days['special_days'] = 1
special_days = special_days.drop(special_days[special_days.date.duplicated()].index)

national = special_days[special_days.locale == 'National']
local = special_days[special_days.locale == 'Local']
regional = special_days[special_days.locale == 'Regional']

# national_holiday_events -> train, test
national = national.drop(['locale', 'locale_name'], axis=1)
train = pd.merge(train, national, on='date', how='left')
test = pd.merge(test, national, on='date', how='left')

# regional_holiday_events -> train, test
for i, j in zip(regional.date, regional.locale_name):
    train['special_days'][(train.date == i) & (train.state == j)] = 1

for i, j in zip(regional.date, regional.locale_name):
    test['special_days'][(test.date == i) & (test.state == j)] = 1

# local_holiday_events -> train, test
for i, j in zip(local.date, local.locale_name):
    train['special_days'][(train.date == i) & (train.city == j)] = 1

for i, j in zip(local.date, local.locale_name):
    test['special_days'][(test.date == i) & (test.city == j)] = 1

In [119]:
test.head()

,id,date,store_nbr,family,onpromotion,oil_price,city,state,type,cluster,special_days
0,3000888,2017-08-16,1,AUTOMOTIVE,0,46.8,Quito,Pichincha,D,13,NaN
1,3000889,2017-08-16,1,BABY CARE,0,46.8,Quito,Pichincha,D,13,NaN
2,3000890,2017-08-16,1,BEAUTY,2,46.8,Quito,Pichincha,D,13,NaN
3,3000891,2017-08-16,1,BEVERAGES,20,46.8,Quito,Pichincha,D,13,NaN
4,3000892,2017-08-16,1,BOOKS,0,46.8,Quito,Pichincha,D,13,NaN


In [120]:
# fill missing values
train.isnull().sum()

id                    0
date                  0
store_nbr             0
family                0
sales                 0
onpromotion           0
oil_price        928422
city                  0
state                 0
type                  0
cluster               0
special_days    2766390
dtype: int64

In [121]:
test.isnull().sum()

id                  0
date                0
store_nbr           0
family              0
onpromotion         0
oil_price        7128
city                0
state               0
type                0
cluster             0
special_days    28446
dtype: int64

In [122]:
oil.head(10)

,date,oil_price
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
5,2013-01-08,93.21
6,2013-01-09,93.08
7,2013-01-10,93.81
8,2013-01-11,93.60
9,2013-01-14,94.27


In [123]:
# Fill NaN in oil_price as oil price does not fluctuate
train['oil_price'] = train['oil_price'].fillna(method='bfill')
test['oil_price'] = test['oil_price'].fillna(method='bfill')

In [124]:
train.isnull().sum()

id                    0
date                  0
store_nbr             0
family                0
sales                 0
onpromotion           0
oil_price             0
city                  0
state                 0
type                  0
cluster               0
special_days    2766390
dtype: int64

In [125]:
test.isnull().sum()

id                  0
date                0
store_nbr           0
family              0
onpromotion         0
oil_price           0
city                0
state               0
type                0
cluster             0
special_days    28446
dtype: int64

In [126]:
train.head()

,id,date,store_nbr,family,sales,onpromotion,oil_price,city,state,type,cluster,special_days
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,93.14,Quito,Pichincha,D,13,1.0
1,1,2013-01-01,1,BABY CARE,0.0,0,93.14,Quito,Pichincha,D,13,1.0
2,2,2013-01-01,1,BEAUTY,0.0,0,93.14,Quito,Pichincha,D,13,1.0
3,3,2013-01-01,1,BEVERAGES,0.0,0,93.14,Quito,Pichincha,D,13,1.0
4,4,2013-01-01,1,BOOKS,0.0,0,93.14,Quito,Pichincha,D,13,1.0


In [127]:
# Fill NaN for special_days with 0
train['special_days'] = train['special_days'].fillna(0)
test['special_days'] = test['special_days'].fillna(0)

In [128]:
train.isnull().sum()

id              0
date            0
store_nbr       0
family          0
sales           0
onpromotion     0
oil_price       0
city            0
state           0
type            0
cluster         0
special_days    0
dtype: int64

In [129]:
test.isnull().sum()

id              0
date            0
store_nbr       0
family          0
onpromotion     0
oil_price       0
city            0
state           0
type            0
cluster         0
special_days    0
dtype: int64

In [130]:
# set index
train = train.set_index('id')
test = test.set_index('id')

In [131]:
train.head()

,date,store_nbr,family,sales,onpromotion,oil_price,city,state,type,cluster,special_days
id,,,,,,,,,,,
0,2013-01-01,1,AUTOMOTIVE,0.0,0,93.14,Quito,Pichincha,D,13,1.0
1,2013-01-01,1,BABY CARE,0.0,0,93.14,Quito,Pichincha,D,13,1.0
2,2013-01-01,1,BEAUTY,0.0,0,93.14,Quito,Pichincha,D,13,1.0
3,2013-01-01,1,BEVERAGES,0.0,0,93.14,Quito,Pichincha,D,13,1.0
4,2013-01-01,1,BOOKS,0.0,0,93.14,Quito,Pichincha,D,13,1.0


In [132]:
# Drop Useless columns
train = train.drop(columns=['city', 'state', 'cluster', 'store_nbr'], axis=1)
test = test.drop(columns=['city', 'state', 'cluster', 'store_nbr'], axis=1)

In [133]:
train.head()

,date,family,sales,onpromotion,oil_price,type,special_days
id,,,,,,,
0,2013-01-01,AUTOMOTIVE,0.0,0,93.14,D,1.0
1,2013-01-01,BABY CARE,0.0,0,93.14,D,1.0
2,2013-01-01,BEAUTY,0.0,0,93.14,D,1.0
3,2013-01-01,BEVERAGES,0.0,0,93.14,D,1.0
4,2013-01-01,BOOKS,0.0,0,93.14,D,1.0


In [134]:
# Convert Categorical into Numerical
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [135]:
train.head()

,sales,onpromotion,oil_price,special_days,date_2013-01-01,date_2013-01-02,date_2013-01-03,date_2013-01-04,date_2013-01-05,date_2013-01-06,...,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD,type_A,type_B,type_C,type_D,type_E
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0,93.14,1.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0.0,0,93.14,1.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0.0,0,93.14,1.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0.0,0,93.14,1.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0.0,0,93.14,1.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [136]:
# Correlation b/w dependent and independent variables
# train.corr()['sales'].sort_values(ascending=True).head(10)
# train.corr()['sales'].sort_values(ascending=False).head(10)

In [137]:
# Set Independent and Dependent Variable
X = train.drop('sales')

KeyError: "['sales'] not found in axis"